In [1]:
import os
import openai
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser


from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.memory import ConversationBufferMemory
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.text_splitter import CharacterTextSplitter, TokenTextSplitter
load_dotenv()

True

In [2]:
openai.organization = os.getenv("OPENAI_ORG_ID")
openai.api_key = os.getenv("OPENAI_API_KEY")

#### QA SYSTEM



STEPS 
- Load documents
- Split documents into semantic meaninful chunks 



- Create embeddings for documnets
- Store embeddings in vector database (lookup similar vectors)
- Retrieval

In [16]:
llm = ChatOpenAI(temperature=0.0, model='gpt-3.5-turbo')

In [19]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)


In [3]:
file_path = '/Users/Labi/V_env/assistant/QA_system/MLOps.pdf'

In [4]:
loader = PyPDFLoader(file_path=file_path)

In [5]:
docs = loader.load()

In [6]:
text_splitter = TokenTextSplitter(
    # separator="\n",
    chunk_size=1000,
    chunk_overlap=150,
    # length_function=len
)

In [7]:
# splitting documents into small semantic meaningful chunks
pages=text_splitter.split_documents(docs)

In [8]:
embeddings = OpenAIEmbeddings()

In [10]:
persist_directory = '/Users/Labi/V_env/assistant/QA_system/chroma'

In [11]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [12]:
chroma_db = Chroma.from_documents(
    documents=pages,
    embedding=embeddings,
    persist_directory=persist_directory
)

In [13]:
query = 'what are the principles and components of mlops'

In [ ]:
chroma_db.similarity_search(query, k=2)

In [17]:
retriever = chroma_db.as_retriever()

In [20]:
qa_stuff = ConversationalRetrievalChain.from_llm(
    llm=llm, 
    # return_source_documents=True,
    # chain_type="stuff", 
    retriever=retriever, 
    # verbose=True,
    memory=memory
)

In [21]:
response = qa_stuff({'question':query})

In [46]:
response['answer']

'The principles of MLOps identified in the research are as follows:\n\n1. CI/CD automation: Continuous integration, continuous delivery, and continuous deployment to increase productivity.\n2. Workflow orchestration: Coordinating ML workflow tasks according to directed acyclic graphs (DAGs) to manage dependencies.\n3. Reproducibility: Ability to reproduce ML experiments and obtain the same results.\n4. Versioning: Version control for data, model, and code to ensure reproducibility and traceability.\n5. Collaboration: Collaborative work culture and tools to reduce domain silos and foster communication.\n6. Continuous ML training & evaluation: Periodic retraining of ML models based on new data and evaluating model performance.\n7. ML metadata tracking/logging: Tracking and logging metadata for tasks in the ML workflow pipeline to ensure traceability.\n8. Continuous monitoring: Periodically assessing data, model, code, and infrastructure resources for potential errors or changes.\n9. Feed

In [39]:
def agent(
        model_name,
        file, 
        chain_type, 
        persist_directory, 
        k
):
    # load documents
    loader = PyPDFLoader(file)
    load_docs = loader.load()

    # create doc chunks
    text_splitter = TokenTextSplitter(
        chunk_size=1000,
        chunk_overlap=150,
    )
    docs = text_splitter.split_documents(load_docs)

    # define embedding
    embeddings = OpenAIEmbeddings()

    # create vector database from data
    vector_db = Chroma.from_documents(
        documents=docs,
        embedding=embeddings,
        persist_directory=persist_directory
    )

    # define retriever
    retriever = vector_db.as_retriever(
        search_type="similarity", 
        search_kwargs={"k": k}
    )

    memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

    # create a chatbot chain. Memory is managed externally.
    qa = ConversationalRetrievalChain.from_llm(
        llm=ChatOpenAI(
            model_name=model_name, 
            temperature=0
        ), 
        chain_type=chain_type, 
        retriever=retriever, 
        memory=memory
    )
    return qa 


In [51]:
def respond(
        agent
):
    chat = True
    while chat:
        query = input()
        if query == 'exit':
            chat = False
        else:
            response=agent({'question':query})
            print(response['answer'])
            print('\n')
    

In [52]:
resp = respond(
    agent=agent(
        model_name='gpt-3.5-turbo',
        file=file_path,
        chain_type='stuff',
        persist_directory=persist_directory,
        k=2
    )
)